# Задание к занятию «Рекомендации на основе скрытых факторов»



Описание задания:

Попробовать разные модели из пакета Implicit, чтобы получить на тестовой выборке precision@50 > 0.1

 

Какие данные

Датасет lastfm-360K.


## Подготовка виртуальной машины

Устанавливаем CUDA, cобираем implicit с поддержкой gpu

In [0]:
#!kill -9 -1

In [0]:
#!wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh &&  /bin/bash ~/miniconda.sh -b -p ./conda &&  rm ~/miniconda.sh 
#!echo ". ~/conda/etc/profile.d/conda.sh" >> ~/.bashrc && echo "conda activate base" >> ~/.bashrc  
#!conda update conda

In [6]:
!cat /etc/lsb-release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=17.10
DISTRIB_CODENAME=artful
DISTRIB_DESCRIPTION="Ubuntu 17.10"


In [17]:
# нужна восьмая CUDA, потому что в Collaboratory более старый драйвер видеокарт (версии 384, который несовместим с CUDA 9)
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

--2018-06-12 17:10:03--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?qR0axdcf8tlAAyEWHvm66FRPhVxgcvTtuJPblI6oh68xtPOo5mW6QQ5lFLXM1FG7webKNNrp4DkX3bsepg0lBG6atuvKf_1cE2qRUdh2DxdiDKzFQ9rwmeTj4zXbEcsadlxVJD4lTwU7gGzDqacObYIwD2my6fQWRF2A5q_PMTMSCmSczHEYBdu0V-JqjujAb5gcjHTg3eYE6yGu8XiBtygMrw [following]
--2018-06-12 17:10:03--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?qR0axdcf8tlAAyEWHvm66FRPhVxgcvTtuJPblI6oh68xtPOo5mW6QQ5lFLXM1FG7webKNNrp4DkX3bsepg0lBG6atuvK

cuda-repo-ubuntu160 100%[===================>]   1.78G   147MB/s    in 12s     

2018-06-12 17:10:16 (147 MB/s) - ‘cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb.2’ saved [1913589814/1913589814]



In [18]:
# делаем по инструкции с сайта NVidia
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

(Reading database ... 82771 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb ...
Unpacking cuda-repo-ubuntu1604-8-0-local-ga2 (8.0.61-1) over (8.0.61-1) ...
Setting up cuda-repo-ubuntu1604-8-0-local-ga2 (8.0.61-1) ...
Note: Check first if apt-key functionality is needed at all - it probably isn't!
OK


In [13]:
!apt-get update

Get:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Ign:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Hit:4 http://security.ubuntu.com/ubuntu artful-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu artful InRelease
Hit:6 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Reading package lists... Done


In [0]:
#!apt --fix-broken install

In [14]:
!apt-get install cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cuda is already the newest version (8.0.61-1).
You might want to run 'apt --fix-broken install' to correct these.
The following packages have unmet dependencies:
 cuda-toolkit-8-0 : Depends: cuda-visual-tools-8-0 (>= 8.0.61) but it is not going to be installed
E: Unmet dependencies. Try 'apt --fix-broken install' with no packages (or specify a solution).


In [20]:
# скачиваем патч
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/patches/2/cuda-repo-ubuntu1604-8-0-local-cublas-performance-update_8.0.61-1_amd64-deb


--2018-06-12 17:17:01--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/patches/2/cuda-repo-ubuntu1604-8-0-local-cublas-performance-update_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/patches/2/cuda-repo-ubuntu1604-8-0-local-cublas-performance-update_8.0.61-1_amd64.deb?qLgSRbYYZAcVOgq__bWIolfTr33-DchtNZkM5dlM5LvJgxZ5KuFD5tuXt7llnpzLb7iPJAYzS7iauFt2VOck8Rph9sTQNpwUfAAVAh80B9rDsHx-RbyUIofWdHXm7tyJwmbM-OZ2W4uy5wvCBnpNfTY06gwhHxOugIWyk8u39HHyxMJOGYMWAgNznczLu6gK3gx-2aAZZxCxC_pKTG9u3NzmkRrqv-7YrdomS5RuXE-qZw [following]
--2018-06-12 17:17:01--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/patches/2/cuda-repo-ubuntu1604-8-0-local-cublas-performance-update_8.0.61-1_amd64.deb?qLgSRbYYZAcVOgq__bWIolfTr33

In [21]:
  # устанавливаем патч
!dpkg -i cuda-repo-ubuntu1604-8-0-local-cublas-performance-update_8.0.61-1_amd64-deb

Selecting previously unselected package cuda-repo-ubuntu1604-8-0-local-cublas-performance-update.
(Reading database ... 85600 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1604-8-0-local-cublas-performance-update_8.0.61-1_amd64-deb ...
Unpacking cuda-repo-ubuntu1604-8-0-local-cublas-performance-update (8.0.61-1) ...
Setting up cuda-repo-ubuntu1604-8-0-local-cublas-performance-update (8.0.61-1) ...
Note: Check first if apt-key functionality is needed at all - it probably isn't!
OK


In [23]:
!nvidia-smi

Tue Jun 12 17:17:38 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    25W / 149W |      1MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [24]:
# нужно для совсемстимости CUDA-8 и implicit; в докере стоит gcc версии 7 - слишком новый
!apt-get install gcc-5 g++-5
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cpp-5 gcc-5-base libasan2 libgcc-5-dev libmpx0 libstdc++-5-dev
Suggested packages:
  gcc-5-locales g++-5-multilib gcc-5-doc libstdc++6-5-dbg gcc-5-multilib
  libgcc1-dbg libgomp1-dbg libitm1-dbg libatomic1-dbg libasan2-dbg
  liblsan0-dbg libtsan0-dbg libubsan0-dbg libcilkrts5-dbg libmpx0-dbg
  libquadmath0-dbg libstdc++-5-doc
The following NEW packages will be installed:
  cpp-5 g++-5 gcc-5 gcc-5-base libasan2 libgcc-5-dev libmpx0 libstdc++-5-dev
0 upgraded, 8 newly installed, 0 to remove and 17 not upgraded.
Need to get 28.6 MB of archives.
After this operation, 98.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 gcc-5-base amd64 5.5.0-1ubuntu2 [17.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 cpp-5 amd64 5.5.0-1ubuntu2 [7,798 kB]
Get:3 http://archive.u

In [25]:
# скачиваем код пакета
!git clone https://github.com/benfred/implicit/
!cd implicit; git pull

Cloning into 'implicit'...
remote: Counting objects: 874, done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 874 (delta 14), reused 12 (delta 5), pack-reused 843
Receiving objects: 100% (874/874), 1.89 MiB | 21.51 MiB/s, done.
Resolving deltas: 100% (545/545), done.
Already up-to-date.


In [26]:
# собираем implicit с поддержко CUDA
!export HOST_COMPILER=gcc-5; export CUDAHOME=/usr/local/cuda; cd implicit; python3 setup.py install

running install
running bdist_egg
running egg_info
creating implicit.egg-info
writing implicit.egg-info/PKG-INFO
writing dependency_links to implicit.egg-info/dependency_links.txt
writing requirements to implicit.egg-info/requires.txt
writing top-level names to implicit.egg-info/top_level.txt
writing manifest file 'implicit.egg-info/SOURCES.txt'
reading manifest file 'implicit.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'implicit.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/tests
copying tests/als_test.py -> build/lib.linux-x86_64-3.6/tests
copying tests/knn_test.py -> build/lib.linux-x86_64-3.6/tests
copying tests/bpr_test.py -> build/lib.linux-x86_64-3.6/tests
copying tests/recommender_base_test.py -> build/lib.linux-x86_64-3.6/tests
copying tests/__init__.py -> build/lib.linux-x86_64-3.6/t

/usr/local/cuda/bin/nvcc -I/usr/local/cuda/include -I. -I/usr/include/python3.6m -c implicit/cuda/bpr.cu -o build/temp.linux-x86_64-3.6/implicit/cuda/bpr.o -gencode=arch=compute_30,code=sm_30 -gencode=arch=compute_50,code=sm_50 -gencode=arch=compute_60,code=sm_60 -gencode=arch=compute_60,code=compute_60 --ptxas-options=-v -O2 -c --compiler-options '-fPIC'
ptxas info    : 312 bytes gmem
ptxas info    : Compiling entry function '_ZN8implicit17bpr_update_kernelEiPjS0_PiS1_S1_iPfS2_ffbS1_' for 'sm_30'
ptxas info    : Function properties for _ZN8implicit17bpr_update_kernelEiPjS0_PiS1_S1_iPfS2_ffbS1_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 30 registers, 144 bytes smem, 416 bytes cmem[0], 32 bytes cmem[2]
ptxas info    : 312 bytes gmem
ptxas info    : Compiling entry function '_ZN8implicit17bpr_update_kernelEiPjS0_PiS1_S1_iPfS2_ffbS1_' for 'sm_50'
ptxas info    : Function properties for _ZN8implicit17bpr_update_kernelEiPjS0_PiS1_S1_iPfS2_ffbS1_


In [1]:
# чтобы дальше работать с пакетом, нужно выбрать в меню "Runtime->Restart runtime..."
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os

os.environ["OPENBLAS_NUM_THREADS"] = "1"
als = AlternatingLeastSquares(use_gpu=True)
item_users = csr_matrix([[1, 2], [3, 4]])
als.fit(item_users)

GPU training requires factor size to be a multiple of 32. Increasing factors from 100 to 128.
100%|██████████| 15.0/15 [00:00<00:00, 952.00it/s]


## Загрузка и обработка данных

In [0]:
#from google.colab import files
#uploaded = files.upload()
#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(
#      name=fn, length=len(uploaded[fn])))

In [0]:
#!ls -l

In [0]:
#from scipy.sparse import load_npz

# загружаем заранее подготовленные данные
#user_item_matrix = load_npz("./user_item_lastfm.npz")

In [0]:
#user_item_matrix

In [2]:
#воспользуемся датасетом lastfm из implicit в CSR формате
from implicit.datasets.lastfm import get_lastfm
artists, users, plays = get_lastfm()

184MB [00:07, 23.8MB/s]                           


In [3]:
len(artists), len(users)

(292385, 358868)

In [4]:
plays

<292385x358868 sparse matrix of type '<class 'numpy.float32'>'
	with 17535606 stored elements in Compressed Sparse Row format>

In [0]:
#т.к. CSR формат не поддерживает fancy индексирование, то пеереведем в COO и транспонируем для дальнейшего разбиения на обучающую и тестовые матрицы
user_item_matrix = plays.tocoo().T

In [0]:
#освободим память
del artists, users, plays

In [7]:
user_item_matrix

<358868x292385 sparse matrix of type '<class 'numpy.float32'>'
	with 17535606 stored elements in COOrdinate format>

In [0]:
import numpy as np

# делим разреженную матрицу на обучающую и тестовую
total_len = user_item_matrix.data.size
train_len = int(total_len * 0.8)
all_indices = np.arange(total_len)
np.random.seed(42)
train_indices = np.random.choice(all_indices, train_len, replace=False)
train_mask = np.in1d(all_indices, train_indices)

In [9]:
print(all_indices)
print(train_indices)
print(train_mask)
print(len(train_mask))

[       0        1        2 ... 17535603 17535604 17535605]
[ 2169573  6534920 15970232 ...   466581 15749026 15114794]
[ True False  True ... False  True  True]
17535606


In [0]:
from scipy.sparse import coo_matrix

def get_masked(arr, mask):
    return coo_matrix(
        (
            [np.float32(item) for item in arr.data[mask]],
            (arr.row[mask], arr.col[mask])
        ),
        arr.shape
    )

In [0]:
train_csr = get_masked(user_item_matrix, train_mask).tocsr()
# для обучения нужна item*user матрица
train = train_csr.T
test_coo = get_masked(user_item_matrix, ~train_mask)
test_csr = test_coo.tocsr()

## Модели

Для сравнения времени обучения и предсказания используем одинаковые параметры ALS.


### ALS

In [0]:
os.environ["OPENBLAS_NUM_THREADS"] = "1"
#Параметры ALS
n_factors = 256
n_iterations=15
n_users = 1000
regularization=0.2


In [0]:
users = list(set(test_coo.row))
users = users[:n_users] #уменьшим количество пользователей на тестовой выборке

In [0]:
def hitrate(k, recs, users):
    hits = 0
    for user in users:
        if recs[user]:
            rec_items, _ = zip(*recs[user])
            hits += len(set(rec_items[:k]).intersection(set(test_csr[user].indices))) > 0
    return hits / len(users)
  
def precision(k, recs, users):
  num_users = 0
  sum_user_prec = .0
  for user in users:
    if recs[user]:
      num_users += 1
      rec_items, _ = zip(*recs[user])
      user_prec = float(len(set(rec_items[:k]).intersection(set(test_csr[user].indices)))) / k
      sum_user_prec += user_prec
  return sum_user_prec / num_users 

def get_recs(users, model):
    return {
        user: model.recommend(userid=user, user_items=train_csr, N=50)
        for user in users
    }

In [0]:
%%time

als = AlternatingLeastSquares(
    factors=n_factors,
    iterations=n_iterations,
    regularization=regularization,
    use_gpu=True
)
als.fit(train)

# посчитаем по 50 рекомендаций для каждого пользователя из тестовой выборки
recs = get_recs(users, als)
  




  0%|          | 0/15 [00:00<?, ?it/s]

  3%|▎         | 0.5/15 [00:02<01:25,  5.93s/it]

  7%|▋         | 1.0/15 [00:05<01:22,  5.91s/it]

 10%|█         | 1.5/15 [00:08<01:18,  5.84s/it]

 13%|█▎        | 2.0/15 [00:11<01:16,  5.86s/it]

 17%|█▋        | 2.5/15 [00:14<01:12,  5.80s/it]

 20%|██        | 3.0/15 [00:17<01:10,  5.85s/it]

 23%|██▎       | 3.5/15 [00:20<01:06,  5.80s/it]

 27%|██▋       | 4.0/15 [00:23<01:04,  5.84s/it]

 30%|███       | 4.5/15 [00:26<01:00,  5.79s/it]

 33%|███▎      | 5.0/15 [00:29<00:58,  5.82s/it]

 37%|███▋      | 5.5/15 [00:31<00:54,  5.78s/it]

 40%|████      | 6.0/15 [00:34<00:52,  5.83s/it]

 43%|████▎     | 6.5/15 [00:37<00:49,  5.81s/it]

 47%|████▋     | 7.0/15 [00:40<00:46,  5.85s/it]

 50%|█████     | 7.5/15 [00:43<00:43,  5.83s/it]

 53%|█████▎    | 8.0/15 [00:46<00:40,  5.85s/it]

 57%|█████▋    | 8.5/15 [00:49<00:37,  5.82s/it]

 60%|██████    | 9.0/15 [00:52<00:35,  5.86s/it]

 63%|██████▎   | 9.5/15 [00:55<00:32,  5.82s/it]

 67%|███

In [121]:
print("hitrate@50:", hitrate(50, recs, users))
print("precision@50:", precision(50, recs, users))

hitrate@50: 0.94
precision@50: 0.06719999999999997


### ALS Annoy

In [0]:
!pip install annoy

In [0]:
from implicit.approximate_als import AnnoyAlternatingLeastSquares


In [84]:
%%time


als_annoy = AnnoyAlternatingLeastSquares(
    factors=n_factors,
    iterations=n_iterations,
    regularization=regularization
)
als_annoy.fit(train)

# посчитаем по 50 рекомендаций для каждого пользователя из тестовой выборки
recs_annoy = get_recs(users, als_annoy)





  0%|          | 0/30 [00:00<?, ?it/s]

  2%|▏         | 0.5/30 [00:02<02:26,  4.96s/it]

  3%|▎         | 1.0/30 [00:05<02:25,  5.01s/it]

  5%|▌         | 1.5/30 [00:07<02:20,  4.92s/it]

  7%|▋         | 2.0/30 [00:10<02:19,  5.00s/it]

  8%|▊         | 2.5/30 [00:12<02:15,  4.92s/it]

 10%|█         | 3.0/30 [00:14<02:14,  4.99s/it]

 12%|█▏        | 3.5/30 [00:17<02:10,  4.93s/it]

 13%|█▎        | 4.0/30 [00:19<02:09,  5.00s/it]

 15%|█▌        | 4.5/30 [00:22<02:05,  4.94s/it]

 17%|█▋        | 5.0/30 [00:24<02:05,  5.01s/it]

 18%|█▊        | 5.5/30 [00:27<02:01,  4.95s/it]

 20%|██        | 6.0/30 [00:29<02:00,  5.03s/it]

 22%|██▏       | 6.5/30 [00:32<01:56,  4.96s/it]

 23%|██▎       | 7.0/30 [00:34<01:55,  5.03s/it]

 25%|██▌       | 7.5/30 [00:37<01:51,  4.96s/it]

 27%|██▋       | 8.0/30 [00:39<01:50,  5.04s/it]

 28%|██▊       | 8.5/30 [00:42<01:46,  4.97s/it]

 30%|███       | 9.0/30 [00:44<01:46,  5.05s/it]

 32%|███▏      | 9.5/30 [00:47<01:42,  4.98s/it]

 33%|███

 57%|█████▋    | 17.0/30 [01:25<01:06,  5.10s/it]

 58%|█████▊    | 17.5/30 [01:27<01:02,  5.03s/it]

 60%|██████    | 18.0/30 [01:30<01:01,  5.10s/it]

 62%|██████▏   | 18.5/30 [01:32<00:57,  5.04s/it]

 63%|██████▎   | 19.0/30 [01:35<00:56,  5.10s/it]

 65%|██████▌   | 19.5/30 [01:38<00:52,  5.04s/it]

 67%|██████▋   | 20.0/30 [01:40<00:51,  5.11s/it]

 68%|██████▊   | 20.5/30 [01:43<00:47,  5.04s/it]

 70%|███████   | 21.0/30 [01:45<00:45,  5.09s/it]

 72%|███████▏  | 21.5/30 [01:48<00:42,  5.03s/it]

 73%|███████▎  | 22.0/30 [01:50<00:40,  5.09s/it]

 75%|███████▌  | 22.5/30 [01:53<00:37,  5.02s/it]

 77%|███████▋  | 23.0/30 [01:55<00:35,  5.08s/it]

 78%|███████▊  | 23.5/30 [01:58<00:32,  5.01s/it]

 80%|████████  | 24.0/30 [02:00<00:30,  5.07s/it]

 82%|████████▏ | 24.5/30 [02:03<00:27,  5.00s/it]

 83%|████████▎ | 25.0/30 [02:05<00:25,  5.07s/it]

 85%|████████▌ | 25.5/30 [02:08<00:22,  5.00s/it]

 87%|████████▋ | 26.0/30 [02:10<00:20,  5.09s/it]

 88%|████████▊ | 26.5/30 [02:13

CPU times: user 2min 55s, sys: 1min 52s, total: 4min 47s
Wall time: 4min 47s


In [85]:
print("hitrate@50:", hitrate(50, recs_annoy, users))
print("precision@50:", precision(50, recs_annoy, users))

CPU times: user 1.15 s, sys: 32 ms, total: 1.18 s
Wall time: 1.19 s


### ALS NMSLib

In [0]:
!pip install nmslib

In [0]:
from implicit.approximate_als import NMSLibAlternatingLeastSquares

In [56]:
%%time

als_nms = NMSLibAlternatingLeastSquares(
    factors=n_factors,
    iterations=n_iterations,
    regularization=regularization
)
als_nms.fit(train)

# посчитаем по 50 рекомендаций для каждого пользователя из тестовой выборки
recs_nms = get_recs(users, als_nms)

GPU training requires factor size to be a multiple of 32. Increasing factors from 100 to 128.
100%|██████████| 15.0/15 [00:38<00:00,  2.63s/it]


CPU times: user 10min 8s, sys: 29.4 s, total: 10min 38s
Wall time: 5min 42s


In [58]:
print("hitrate@50:", hitrate(50, recs_nms, users))
print("precision@50:", precision(50, recs_nms, users))


hitrate@50: 0.58305
precision@50: 0.03145699999999819
CPU times: user 5.23 s, sys: 70 ms, total: 5.3 s
Wall time: 5.22 s


### ALS Faiss

In [0]:
!wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda8.0.61_2.tar.bz2
!tar xvjf faiss-gpu-1.2.1-py36_cuda8.0.61_2.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [0]:
from implicit.approximate_als import FaissAlternatingLeastSquares

In [61]:
%%time

# обучимся на дефолтных параметрах
als_faiss = FaissAlternatingLeastSquares(
    factors=n_factors,
    iterations=n_iterations,
    regularization=regularization
    use_gpu=True
)
als_faiss.fit(train)
%%time
# посчитаем по 50 рекомендаций для каждого пользователя из тестовой выборки
recs_faiss = get_recs(users, als_faiss)

GPU training requires factor size to be a multiple of 32. Increasing factors from 100 to 128.
100%|██████████| 15.0/15 [00:38<00:00,  2.61s/it]


CPU times: user 51.1 s, sys: 30.2 s, total: 1min 21s
Wall time: 1min 21s


In [63]:

print("hitrate@50:", hitrate(50, recs_faiss, users))
print("precision@50:", precision(50, recs_faiss, users))

hitrate@50: 0.68625
precision@50: 0.030159999999997637
